<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/Dataset_making/Train_YOLOX_on_a_Custom_Dataset_YouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Train YOLOX on Custom Objects

This tutorial is based on the [YOLOX repository](https://github.com/Megvii-BaseDetection/YOLOX) by [the Megvii Team](https://github.com/Megvii-BaseDetection). This notebook shows training on **your own custom objects**. Many thanks to the Megvii Team for putting this repository together - we hope that in combination with clean data management tools at Roboflow, this technologoy will become easily accessible to any developer wishing to use computer vision in their projects.

### Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on [How to Train YOLOX](blog.roboflow.com/how-to-train-yolox-on-a-custom-dataset/), concurrently.

### Steps Covered in this Tutorial

In this tutorial, we will walk through the steps required to train YOLOR on your custom objects. We use a [public blood cell detection dataset](https://public.roboflow.ai/object-detection/bccd), which is open source and free to use. You can also use this notebook on your own data. We will use Roboflow to preprocess our images.

To train our detector we take the following steps:

* Install YOLOX dependencies
* Download and Prepare custom YOLOX object detection data
* Download Pre-Trained Weights for YOLOX
* Run YOLOX training
* Evaluate YOLOX performance
* Run YOLOX inference on test images
* Export saved YOLOX weights for future inference

### **About**

[Roboflow](https://roboflow.com) enables teams to deploy custom computer vision models quickly and accurately. Convert data from to annotation format, assess dataset health, preprocess, augment, and more. It's free for your first 1000 source images.

**Looking for a vision model available via API without hassle? Try Roboflow Train.**

![Roboflow Wordmark](https://i.imgur.com/dcLNMhV.png)

# Install YOLOX Dependencies

In [1]:
!git clone https://github.com/roboflow-ai/YOLOX.git
%cd YOLOX
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .
!pip uninstall -y torch torchvision torchaudio
# May need to change in the future if Colab no longer uses CUDA 11.0
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'YOLOX'...
remote: Enumerating objects: 798, done.
remote: Total 798 (delta 0), reused 0 (delta 0), pack-reused 798
Receiving objects: 100% (798/798), 5.78 MiB | 18.73 MiB/s, done.
Resolving deltas: 100% (421/421), done.
/content/YOLOX
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement onnxruntime==1.8.0 (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.17.0, 1.17.1, 1.17.3)
ERROR: No matching distribution found for onnxruntime==1.8.0
U

## Install Nvidia Apex

In [2]:
%cd /content/
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/content
Cloning into 'apex'...
remote: Enumerating objects: 11703, done.
remote: Counting objects: 100% (3796/3796), done.
remote: Compressing objects: 100% (626/626), done.
remote: Total 11703 (delta 3410), reused 3316 (delta 3166), pack-reused 7907
Receiving objects: 100% (11703/11703), 15.53 MiB | 19.73 MiB/s, done.
Resolving deltas: 100% (8212/8212), done.
/content/apex
Using pip 24.0 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 24.2 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /content/apex
  Running command pip subprocess to install build dependencies
  Using pip 24.0 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.7 MB/s eta 0:00:00
    Getting requirements to build wheel: started
    Running comma

## Install PyCocoTools

In [3]:
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8wojm1v9
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8wojm1v9
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=375616 sha256=6a20782e802becaebd5d50209042b48d9e7fe063432c349d98b61978a350208b
  Stored in directory: /tmp/pip-ephem-wheel-cache-74ox3_ia/wheels/39/61/b4/480fbddb4d3d6bc34083e7397bc6f5d1381f79acc68e9f3511
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.7
    Uninstalling pycocotools-2.0.7:
      Successfully uninstalled pycocotools-2.0.7


In [5]:
!pip install youtube-search-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [7]:
from youtubesearchpython import SearchVideos

# Initialize the SearchVideos object with your search query
search = SearchVideos("Iran moral patrol", offset=1, mode="json", max_results=20)

# Retrieve the search results
results = search.result()

# Check if the results are in the expected format
if isinstance(results, dict) and 'items' in results:
    # Prepare a list to hold the video URLs
    video_urls = []

    # Iterate through the results and extract the video URLs
    for item in results['items']:
        try:
            video_urls.append(item['link'])
        except KeyError:
            print(f"Error accessing 'link' in item: {item}")
            continue

    # Write the video URLs to a CSV file
    with open('video_urls.csv', 'w', encoding='utf-8') as csvfile:
        for url in video_urls:
            csvfile.write("%s\n" % url)

    print("Video URLs saved to video_urls.csv.")
else:
    print("Results are not in the expected format:", results)

Results are not in the expected format: {
    "search_result": [
        {
            "index": 0,
            "id": "PqMkkuQ6NOc",
            "link": "https://www.youtube.com/watch?v=PqMkkuQ6NOc",
            "title": "Iran's morality police resume hijab street patrols",
            "channel": "Reuters",
            "duration": "1:08",
            "views": 57951,
            "thumbnails": [
                "https://img.youtube.com/vi/PqMkkuQ6NOc/default.jpg",
                "https://img.youtube.com/vi/PqMkkuQ6NOc/hqdefault.jpg",
                "https://img.youtube.com/vi/PqMkkuQ6NOc/mqdefault.jpg",
                "https://img.youtube.com/vi/PqMkkuQ6NOc/sddefault.jpg",
                "https://img.youtube.com/vi/PqMkkuQ6NOc/maxresdefault.jpg"
            ],
            "channeId": "UChqUTb7kYRX8-EiaN3XFrSQ",
            "publishTime": "9 months ago"
        },
        {
            "index": 1,
            "id": "JZCr1l6sDAU",
            "link": "https://www.youtube.com/watch?v=JZC

In [8]:
!pip install video2dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 9.1 MB/s eta 0

In [ ]:
!video2dataset --url_list="video_urls.csv" --url_col="url" --caption_col="caption" --output_folder="dataset"

Starting the downloading of this file
Sharding file number 1 of 1 called /content/apex/videos.csv
0it [00:00, ?it/s]

# Download your Data

We'll download our dataset from Roboflow. Use the "**Pascal VOC**" export format.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).


In [4]:
#to get your roboflow code below please follow the link output by this cell
!pip -q install roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="voc", notebook="yolox")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.1 requires torch>=2, which is not installed.
torchtext 0.17.1 requires torch==2.2.1, which is not installed.


RuntimeError: API Key is of Incorrect Type 
 Expected Type: <class 'str'>
 Input Type: <class 'NoneType'>

In [ ]:
%cd /content/
#from roboflow import Roboflow
#rf = Roboflow(api_key="YOUR_API_KEY", model_format="voc")
#project = rf.workspace().project("YOUR_PROJECT")
#dataset = project.version("YOUR_VERSION").download("voc")

In [ ]:
%cd YOLOX/
!ln -s {dataset.location}/train/ ./datasets/VOCdevkit

## Format Your Data Appropriately

In [ ]:
%mkdir "/content/YOLOX/datasets/VOCdevkit/VOC2007"
!python3 voc_txt.py "/content/YOLOX/datasets/VOCdevkit/"
%mkdir "/content/YOLOX/datasets/VOCdevkit/VOC2012"
!cp -r "/content/YOLOX/datasets/VOCdevkit/VOC2007/." "/content/YOLOX/datasets/VOCdevkit/VOC2012"

## Change the Classes
Make sure you change the classes based on what your dataset. To ensure that the training process will function as intended, write the classes in lowercase with no whitespace.

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/YOLOX/yolox/data/datasets/voc_classes.py

VOC_CLASSES = (
  "rbc",
  "wbc",
  "platelets"
)

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/YOLOX/yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
  "rbc",
  "wbc",
  "platelets"
)

Set the number of classes you have in your dataset in te `NUM_CLASSES` variable

In [ ]:
NUM_CLASSES = 3
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "/content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py"

# Download Pretrained Weights

In [ ]:
%cd /content/
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth
%cd /content/YOLOX/

# Train the Model

In [ ]:
!python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -d 1 -b 16 --fp16 -o -c /content/yolox_s.pth

# Evaluate the Model

In [ ]:
MODEL_PATH = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth.tar"
!python3 tools/eval.py -n  yolox-s -c {MODEL_PATH} -b 64 -d 1 --conf 0.001 -f exps/example/yolox_voc/yolox_voc_s.py

# Test the Model
Make sure you replace the `TEST_IMAGE_PATH` variable with a test image from your dataset

In [ ]:
TEST_IMAGE_PATH = "/content/valid/BloodImage_00057_jpg.rf.1ee93e9ec4d76cfaddaa7df70456c376.jpg"
!python tools/demo.py image -f /content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py -c {MODEL_PATH} --path {TEST_IMAGE_PATH} --conf 0.25 --nms 0.45 --tsize 640 --save_result --device gpu

# Visualize the Predictions
Make sure you replace the `OUTPUT_IMAGE_PATH` with the respective path of the image output. This path can be found somewhere in the `YOLOX_outputs` folder


In [ ]:
from PIL import Image
OUTPUT_IMAGE_PATH = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/vis_res/2021_08_01_19_51_59/BloodImage_00057_jpg.rf.1ee93e9ec4d76cfaddaa7df70456c376.jpg"
Image.open(OUTPUT_IMAGE_PATH)

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp {MODEL_PATH} /content/gdrive/My\ Drive